Title: A Picard-Fuchs Algorithm for Hyperelliptic Level Curves.

Author: Bradley Klee

License: GPLv3. 

In [1]:
\\Input: Polynomial potential function V(x)
\\Output: Matrix encoding of Hermite reduction 
{MatrixG(poly,deg)=
my(Gx,GL,GR);
Gz=matrix(2*deg-1,2*deg-1,j,k,if(j==k&&k<deg,z,0));
GL=matrix(2*deg-1,2*deg-1,j,k,if(0<j-k&&j-k<=deg&&k<deg,-2*polcoef(poly,j-k),0));
GR=matrix(2*deg-1,2*deg-1,j,k,if(k>=deg&&j-(k-deg)>0&&j-(k-deg)<=deg,-(j-(k-deg))*polcoef(poly,j-(k-deg)),0));
Gz+GL+GR};

In [2]:
\\Input: Polynomial potential function V(x)
\\Output: One-form reduction matrices
{ReductionData(poly,deg)=
my(ID,InvG,MU,MV,dMV);
ID=matrix(2*deg-1,2*deg-1,j,k,if(j==k,1,0));
InvG=matinverseimage(MatrixG(poly,deg),ID);
MU=matrix(deg-1,deg-1,j,k,InvG[j,k]);
MV=matrix(deg,deg-1,j,k,InvG[j+deg-1,k]);
dMV=matrix(deg-1,deg-1,j,k,j*MV[j+1,k]);
[MU,dMV,MV]};

In [3]:
\\Input: One-form and One-form reduction matrices
\\Output: Certified, hermite-reduced one form 
{HermiteReduce(dtform,cert,dat,n)=if(n==0,[dtform,cert],
HermiteReduce((dat[1]+1/(2*n-1)*dat[2])*dtform,
cert+1/((2*n-1)*y^(2*n-1))*dat[3]*dtform,
dat,n-1))};

In [4]:
\\n!!
{DoubleFactorial(n) = prod(i=0, (n-1)\2, n - 2*i )};

In [5]:
\\Input: Parity Boolean, one-form reduction matrices
\\Output: Raw Picard-Fuchs data
{PeriodBasis(sym,deg,dat)=
my(dt,bound,reduction);
dt=vector(deg-1,j,if(j==1,1,0))~;
bound=2*if(sym==0,floor((deg-1)/2),floor(deg/4));
decomposition=concat([[dt,vector(deg,j,0)~]],
vector(bound,j,HermiteReduce(((-1/2)^j)*DoubleFactorial(2*j-1)*dt,0,dat,j)));
[vector(bound+1,j,decomposition[j][1]),
vector(bound+1,j,vector(deg,j,x^(j-1))*decomposition[j][2]),
concat([1],vector(bound,j,((-1/(2*y^2))^(j)*DoubleFactorial(2*j-1))))]};

In [6]:
\\Input: Polynomial Potential Function V(x)
\\Output: Processed Picard-Fuchs data
{HyperellipticPicardFuchs(poly)=
my(deg,sym,dat,decomposition,PFCs);
deg=poldegree(poly);
sym=if(subst(poly,x,-x)-poly==0,1,0);
dat = ReductionData(poly,deg);
decomposition = PeriodBasis(sym,deg,dat);
PFCs=lindep(decomposition[1]);
[(1/2)*y^2+poly,decomposition[3]*dt,PFCs,decomposition[2]*PFCs]};

In [7]:
\\Input: Processed Picard-Fuchs data
\\Output: Boolean check value 0
{CheckCertificate(PFData)=
y^(2*length(PFData[2])-2)*(
deriv(PFData[4],x)*deriv(PFData[1],y)
+deriv(PFData[4],y)*deriv(-PFData[1],x)
+PFData[2]*PFData[3]/dt)%(2*PFData[1]-z)
};

Print a few G matrices. 

In [35]:
\\The G Matrix is always invertible. 
coeffs = [a,b,c,d,e,f];
vars = [x,x^2,x^3,x^4,x^5,x^6]~;
MatrixG(coeffs[1..3]*vars[1..3],3)
MatrixG(coeffs[1..4]*vars[1..4],4)
MatrixG(coeffs[1..5]*vars[1..5],5)
MatrixG(coeffs[1..6]*vars[1..6],6)


[   z    0    0    0    0   -a    0    0    0    0    0]

[-2*a    z    0    0    0 -2*b   -a    0    0    0    0]

[-2*b -2*a    z    0    0 -3*c -2*b   -a    0    0    0]

[-2*c -2*b -2*a    z    0 -4*d -3*c -2*b   -a    0    0]

[-2*d -2*c -2*b -2*a    z -5*e -4*d -3*c -2*b   -a    0]

[-2*e -2*d -2*c -2*b -2*a -6*f -5*e -4*d -3*c -2*b   -a]

[-2*f -2*e -2*d -2*c -2*b    0 -6*f -5*e -4*d -3*c -2*b]

[   0 -2*f -2*e -2*d -2*c    0    0 -6*f -5*e -4*d -3*c]

[   0    0 -2*f -2*e -2*d    0    0    0 -6*f -5*e -4*d]

[   0    0    0 -2*f -2*e    0    0    0    0 -6*f -5*e]

[   0    0    0    0 -2*f    0    0    0    0    0 -6*f]


Print a few hypergeometric differential equations.

In [9]:
\\(Cf. OEIS: A000897 )
PFData = HyperellipticPicardFuchs((1/2)*(x^2-(1/4)*x^4));
[PFData[3],CheckCertificate(PFData)]

[[3, 32*z - 16, 16*z^2 - 16*z]~, 0]

In [24]:
\\(Cf. OEIS: A113424 )
PFData = HyperellipticPicardFuchs((1/2)*(x^2-c*x^3));
[-vector(3,j,9/4*(polcoef(PFData[3][j],2,c)*(4/27)+polcoef(PFData[3][j],0,c))),CheckCertificate(PFData)]

[[5, 72*z - 36, 36*z^2 - 36*z], 0]

In [68]:
\\two-parameter elliptic stratum 
PFData = HyperellipticPicardFuchs((1/2)*(x^2+a*x^3+b*x^4));
[PFData[3],CheckCertificate(PFData)]
subst(subst(PFData[3],a,0),b,-1/4)/(-1+2*z-z^2)
pfdat = subst(PFData[3],b,0)/a^4;
-vector(3,j,9/4*(polcoef(pfdat[j],2,a)*4/27+polcoef(pfdat[j],0,a)))

[5, 72*z - 36, 36*z^2 - 36*z]

Examples in Genus 2. 

In [67]:
\\ Parity symmetric sextic (Cf. OEIS: A113424).
\\ 2F1 parameters: (a,b,c) = (1/6,5/6,1).
PFData = HyperellipticPicardFuchs((1/2)*(x^2-(4/27)*x^6));
[-PFData[3],CheckCertificate(PFData)]

[[5*z, 27*z^2 - 9, 9*z^3 - 9*z]~, 0]

In [66]:
\\ Asymmetric sextic.
PFData = HyperellipticPicardFuchs((1/2)*(x^2-x^3-(4/27)*x^6));
[-PFData[3],CheckCertificate(PFData)]
length(PFData[3])

5

Genus-Degree Formulas

In [63]:
\\genus-degree formula: g = floor((d-1)/2) .
\\for d=3,4,5,6... ; g=1,1,2,2...  
v=[];
for(deg=3,8,{
RandCoeffs = vector(deg,j,random(10));
Monomials = vector(deg,j,x^j)~;
PFData = HyperellipticPicardFuchs(RandCoeffs*Monomials+x^deg);
CheckZeros = [a==0|a<-PFData[3]]*vector(length(PFData[3]),j,1)~;
v=concat(v,[[deg,(length(PFData[3])-1)/2,CheckZeros,CheckCertificate(PFData)]]);
})
v

[[3, 1, 0, 0], [4, 1, 0, 0], [5, 2, 0, 0], [6, 2, 0, 0], [7, 3, 0, 0], [8, 3, 0, 0]]

In [65]:
\\apparent genus-degree formula: g' = floor(d/4) .
\\for d=4,6,8,10... ; g'=1,1,2,2...  
v=[];
for(deg=2,7,{
RandCoeffs = vector(deg,j,random(10));
Monomials = vector(deg,j,x^(2*j))~;
PFData = HyperellipticPicardFuchs(RandCoeffs*Monomials+x^(2*deg));
CheckZeros = [a==0|a<-PFData[3]]*vector(length(PFData[3]),j,1)~;
v=concat(v,[[2*deg,(length(PFData[3])-1)/2,CheckZeros,CheckCertificate(PFData)]]);
})
v

[[4, 1, 0, 0], [6, 1, 0, 0], [8, 2, 0, 0], [10, 2, 0, 0], [12, 3, 0, 0], [14, 3, 0, 0]]

Parity symmetry allows quotient decomposition of the Riemann surface into two equal pieces, how neat